In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import importlib

import spacy

from utilities import _support
from utilities._support import ig_profiles

from nltk.tokenize import word_tokenize, sent_tokenize

importlib.reload(_support)

<module 'utilities._support' from 'c:\\Users\\tommaso\\Documents\\_Projects\\IGScraper\\utilities\\_support.py'>

In [2]:
df = pd.read_pickle('gitshared_csv/matteosalviniofficial/matteosalviniofficial_comments_20230316_0-1100_preprocessed.df')

In [3]:
df

,post_id,user,comment,comment_datetime,comment_likes,comment_prep,comment_token,comment_lemma,comment_postag
0,Ck9MRyyKIT5,user_0000027564,"Tutti i cittadini dovrebbero saperlo praticare. Si salverebbero migliaia di vita ogni anno. Questa è civiltà, questa è la società che si evolve. Non con i bagni per chi non é sicuro del proprio genere...",2022-11-14T21:29:33.000Z,29,tutti i cittadini dovrebbero saperlo praticare. si salverebbero migliaia di vita ogni anno. questa è civiltà questa è la società che si evolve. non con i bagni per chi non é sicuro del proprio genere.,"[[tutti, i, cittadini, dovrebbero, saperlo, praticare], [si, salverebbero, migliaia, di, vita, ogni, anno], [questa, è, civiltà, questa, è, la, società, che, si, evolve], [non, con, i, bagni, per, chi, non, é, sicuro, del, proprio, genere]]","[[tutto, il, cittadino, dovere, saperlo, praticare], [si, salvare, migliaio, di, vita, ogni, anno], [questo, essere, civiltà, questo, essere, la, società, che, si, evolvere], [non, con, il, bagno, per, chi, non, é, sicuro, del, proprio, genera]]","[[T, RD, S, VM, V_PC, V], [PC, V, S, E, S, DI, S], [PD, V, S, PD, V, RD, S, PR, PC, V], [BN, E, RD, S, E, PR, BN, V, A, E_RD, AP, S]]"
1,Ck9MRyyKIT5,user_0000019631,"La Professionalità e tempestività di questa ragazza sono encomiabili! Bisognerebbe venissero fatti dei corsi di primo soccorso nelle scuole, sarebbero molto utili!",2022-11-14T21:58:05.000Z,13,la professionalità e tempestività di questa ragazza sono encomiabili bisognerebbe venissero fatti dei corsi di primo soccorso nelle scuole sarebbero molto utili,"[[la, professionalità, e, tempestività, di, questa, ragazza, sono, encomiabili, bisognerebbe, venissero, fatti, dei, corsi, di, primo, soccorso, nelle, scuole, sarebbero, molto, utili]]","[[la, professionalità, e, tempestività, di, questo, ragazza, essere, encomiabile, bisognare, venire, fatto, dio, corso, di, primo, soccorso, nella, scuola, essere, molto, utile]]","[[RD, S, CC, S, E, DD, S, V, A, V, VA, V, E_RD, S, E, NO, S, E_RD, S, V, B, A]]"
2,Ck9MRyyKIT5,user_0000030353,Nessuna correlazione con il siero? Fatevi domande gente! Troppi casi in giro!,2022-11-16T07:00:14.000Z,1,nessuna correlazione con il siero fatevi domande gente troppi casi in giro,"[[nessuna, correlazione, con, il, siero, fatevi, domande, gente, troppi, casi, in, giro]]","[[nessuno, correlazione, con, il, siero, fatevi, domanda, gente, troppo, caso, in, girare]]","[[DI, S, E, RD, S, V, S, S, DI, S, E, S]]"
3,Ck9MRyyKIT5,user_0000028606,Se stata l’angelo di quel l ‘uomo 👏🏻👏🏻👏🏻,2022-11-14T21:35:28.000Z,2,se stata langelo di quel l uomo,"[[se, stata, langelo, di, quel, l, uomo]]","[[se, stare, langelo, di, quello, l, uomo]]","[[CS, V, A, E, PD, RD, S]]"
4,Ck9MRyyKIT5,user_0000055719,Andrebbe insegnato a scuola,2022-11-15T05:08:19.000Z,1,andrebbe insegnato a scuola,"[[andrebbe, insegnato, a, scuola]]","[[andare, insegnare, a, scuola]]","[[VA, V, E, S]]"
...,...,...,...,...,...,...,...,...,...
206355,CiW8UfIMwND,user_0000056164,Mitica Oriana!😍,2022-09-11T08:33:12.000Z,0,mitica oriana,"[[mitica, oriana]]","[[mitico, oriana]]","[[A, A]]"
206356,CiW8UfIMwND,user_0000010176,🙏❤🙏,2022-09-11T08:13:04.000Z,0,,[],[],[]
206357,CiW8UfIMwND,user_0000028852,Esatto 😍,2022-09-11T08:12:32.000Z,0,esatto,[[esatto]],[[esatto]],[[A]]
206358,CiW8UfIMwND,user_0000033569,"Matteo, stai raschiando il fondo del barile?",2022-09-11T08:08:17.000Z,0,matteo stai raschiando il fondo del barile,"[[matteo, stai, raschiando, il, fondo, del, barile]]","[[matteo, stare, raschiare, il, fondere, del, barile]]","[[S, VA, V, RD, S, E_RD, S]]"
